In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6) # Make plots larger by default


In [ ]:
# Upload data
data_path = 'data/secom.data'
labels_path = 'data/secom_labels.data'

# Upload features
features = pd.read_csv(data_path, sep=' ', header=None)

# labels and timestamps
labels = pd.read_csv(labels_path, sep=' ', header=None, names=['label', 'timestamp'])

features.columns = [f'Sensor {i+1}' for i in range(features.shape[1])]

# Combine features and labels
df = pd.concat([features, labels], axis=1)
df = df.drop('timestamp', axis=1)

In [8]:
# check dataset(rows,columns)
print("Dataset Shape:", df.shape)

print("\nFirst 5 Rows:")
print(df.head())

print("\nDataset Info:")
df.info()

print("\nDescriptive Statistics:")
print(df.describe().T)

Dataset Shape: (1567, 591)

First 5 Rows:
   Sensor 1  Sensor 2   Sensor 3   Sensor 4  Sensor 5  Sensor 6  Sensor 7  \
0   3030.93   2564.00  2187.7333  1411.1265    1.3602     100.0   97.6133   
1   3095.78   2465.14  2230.4222  1463.6606    0.8294     100.0  102.3433   
2   2932.61   2559.94  2186.4111  1698.0172    1.5102     100.0   95.4878   
3   2988.72   2479.90  2199.0333   909.7926    1.3204     100.0  104.2367   
4   3032.24   2502.87  2233.3667  1326.5200    1.5334     100.0  100.3967   

   Sensor 8  Sensor 9  Sensor 10  ...  Sensor 582  Sensor 583  Sensor 584  \
0    0.1242    1.5005     0.0162  ...         NaN      0.5005      0.0118   
1    0.1247    1.4966    -0.0005  ...    208.2045      0.5019      0.0223   
2    0.1241    1.4436     0.0041  ...     82.8602      0.4958      0.0157   
3    0.1217    1.4882    -0.0124  ...     73.8432      0.4990      0.0103   
4    0.1235    1.5031    -0.0031  ...         NaN      0.4800      0.4766   

   Sensor 585  Sensor 586  Senso